In [10]:
import numpy as np

In [11]:
ndim = 2
rhs = np.arange(30).reshape((6, 5))
igs = [2, 2]
factor = 100
wall_idx = np.empty((ndim), dtype=object)
for dim in range(ndim):
    wall_idx[dim] = slice(igs[dim], -igs[dim])
print(rhs)
for dim in range(ndim):
    if True:
        for direction in [-1, 1]:
            wall_idx[dim] = (igs[dim] - 1) * direction
            if direction == -1:
                wall_idx[dim] -= 1
            wall_idx_tuple = tuple(wall_idx)
            rhs[wall_idx_tuple] *= factor
            print(wall_idx_tuple)

print(rhs)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]
 [25 26 27 28 29]]
(-2, slice(2, -2, None))
(1, slice(2, -2, None))
(1, -2)
(1, 1)
[[   0    1    2    3    4]
 [   5  600  700  800    9]
 [  10   11   12   13   14]
 [  15   16   17   18   19]
 [  20   21 2200   23   24]
 [  25   26   27   28   29]]


In [12]:
import numba as nb

In [13]:
def stencil_9pt(elem,node,mpv,Sol,ud,diag_inv,dt,coriolis_params):
    igx = elem.igx
    igy = elem.igy

    icxn = node.icx
    icyn = node.icy

    iicxn = icxn - (2 * igx)
    iicyn = icyn - (2 * igy)

    iicxn, iicyn = iicyn, iicxn

    dx = node.dy
    dy = node.dx

    inner_domain = (slice(igx,-igx),slice(igy,-igy))
    i1 = node.i1

    hplusx = mpv.wplus[1][i1].reshape(-1,)
    hplusy = mpv.wplus[0][i1].reshape(-1,)
    hcenter = mpv.wcenter[i1].reshape(-1,)

    diag_inv = diag_inv[i1].reshape(-1,)

    oodx = 1.0 / (dx)
    oody = 1.0 / (dy)

    x_periodic = ud.bdry_type[1] == opts.BdryType.PERIODIC
    y_periodic = ud.bdry_type[0] == opts.BdryType.PERIODIC

    x_wall = ud.bdry_type[1] == opts.BdryType.WALL or ud.bdry_type[1] == opts.BdryType.RAYLEIGH
    y_wall = ud.bdry_type[0] == opts.BdryType.WALL or ud.bdry_type[0] == opts.BdryType.RAYLEIGH
    return lambda p : lap2D_gather(p, igx,igy, iicxn, iicyn, hplusx, hplusy, hcenter, oodx, oody, x_periodic, y_periodic, x_wall, y_wall, diag_inv, coriolis_params)

@nb.jit(nopython=True, nogil=False, cache=True)
def lap2D_gather(p, igx,igy, iicxn, iicyn, hplusx, hplusy, hcenter, oodx, oody, x_periodic, y_periodic, x_wall, y_wall, diag_inv, coriolis):
    ngnc = (iicxn) * (iicyn)
    lap = np.zeros((ngnc))
    cnt_x = 0
    cnt_y = 0

    nine_pt = 0.25 * (2.0) * 1.0
    cyy, cxx, cyx, cxy = coriolis
    oodx2 = 0.5 * oodx**2
    oody2 = 0.5 * oody**2

    for idx in range(iicxn * iicyn):
        ne_topleft = idx - iicxn - 1
        ne_topright = idx - iicxn
        ne_botleft = idx - 1
        ne_botright = idx

        # get indices of the 9pt stencil
        topleft_idx = idx - iicxn - 1
        midleft_idx = idx - 1
        botleft_idx = idx + iicxn - 1

        topmid_idx = idx - iicxn
        midmid_idx = idx
        botmid_idx = idx + iicxn

        topright_idx = idx - iicxn + 1
        midright_idx = idx + 1
        botright_idx = idx + iicxn + 1

        if cnt_x == 0:
            topleft_idx += iicxn - 1
            midleft_idx += iicxn - 1
            botleft_idx += iicxn - 1

            ne_topleft += iicxn - 1
            ne_botleft += iicxn - 1

        if cnt_x == (iicxn - 1):
            topright_idx -= iicxn - 1
            midright_idx -= iicxn - 1
            botright_idx -= iicxn - 1

            ne_topright -= iicxn - 1
            ne_botright -= iicxn - 1

        if cnt_y == 0:
            topleft_idx += ((iicxn) * (iicyn - 1))
            topmid_idx += ((iicxn) * (iicyn - 1))
            topright_idx += ((iicxn) * (iicyn - 1))

            ne_topleft += ((iicxn) * (iicyn - 1))
            ne_topright += ((iicxn) * (iicyn - 1))

        if cnt_y == (iicyn - 1):
            botleft_idx -= ((iicxn) * (iicyn - 1))
            botmid_idx -= ((iicxn) * (iicyn - 1))
            botright_idx -= ((iicxn) * (iicyn - 1))

            ne_botleft -= ((iicxn) * (iicyn - 1))
            ne_botright -= ((iicxn) * (iicyn - 1))

        topleft = p[topleft_idx]
        midleft = p[midleft_idx]
        botleft = p[botleft_idx]

        topmid = p[topmid_idx]
        midmid = p[midmid_idx]
        botmid = p[botmid_idx]

        topright = p[topright_idx]
        midright = p[midright_idx]
        botright = p[botright_idx]

        hplusx_topleft = hplusx[ne_topleft]
        hplusx_botleft = hplusx[ne_botleft]
        hplusy_topleft = hplusy[ne_topleft]
        hplusy_botleft = hplusy[ne_botleft]

        hplusx_topright = hplusx[ne_topright]
        hplusx_botright = hplusx[ne_botright]
        hplusy_topright = hplusy[ne_topright]
        hplusy_botright = hplusy[ne_botright]

        cxx_tl  = cxx[ne_topleft]
        cxx_tr = cxx[ne_topright]
        cxx_bl  = cxx[ne_botleft]
        cxx_br = cxx[ne_botright]

        cxy_tl  = cxy[ne_topleft]
        cxy_tr  = cxy[ne_topright]
        cxy_bl  = cxy[ne_botleft]
        cxy_br  = cxy[ne_botright]

        cyx_tl  = cyx[ne_topleft]
        cyx_tr  = cyx[ne_topright]
        cyx_bl  = cyx[ne_botleft]
        cyx_br  = cyx[ne_botright]

        cyy_tl  = cyy[ne_topleft]
        cyy_tr  = cyy[ne_topright]
        cyy_bl  = cyy[ne_botleft]
        cyy_br  = cyy[ne_botright]

        Dx_tl = 0.5 * (topmid   - topleft + midmid   - midleft) * hplusx_topleft
        Dx_tr = 0.5 * (topright - topmid  + midright - midmid ) * hplusx_topright
        Dx_bl = 0.5 * (botmid   - botleft + midmid   - midleft) * hplusx_botleft
        Dx_br = 0.5 * (botright - botmid  + midright - midmid ) * hplusx_botright

        Dy_tl = 0.5 * (midmid   - topmid   + midleft - topleft) * hplusy_topleft
        Dy_tr = 0.5 * (midright - topright + midmid  - topmid ) * hplusy_topright
        Dy_bl = 0.5 * (botmid   - midmid   + botleft - midleft) * hplusy_botleft
        Dy_br = 0.5 * (botright - midright + botmid  - midmid ) * hplusy_botright

        fac = 1.0
        Dxx = 0.5 * (cxx_tr * Dx_tr - cxx_tl * Dx_tl + cxx_br * Dx_br - cxx_bl * Dx_bl) * oodx * oodx * fac
        Dyy = 0.5 * (cyy_br * Dy_br - cyy_tr * Dy_tr + cyy_bl * Dy_bl - cyy_tl * Dy_tl) * oody * oody * fac
        Dyx = 0.5 * (cyx_br * Dy_br - cyx_bl * Dy_bl + cyx_tr * Dy_tr - cyx_tl * Dy_tl) * oody * oodx * fac
        Dxy = 0.5 * (cxy_br * Dx_br - cxy_tr * Dx_tr + cxy_bl * Dy_bl - cxy_tl * Dx_tl) * oodx * oody * fac


        lap[idx] = Dxx + Dyy + Dyx + Dxy + hcenter[idx] * p[idx]


In [14]:
import numpy as np
import numba as nb

# Define constants for clarity
HALF = 0.5

@nb.jit(nopython=True, nogil=True, cache=True, fastmath=False) # nogil=True should be safe
def lap2D_gather_fused(
    p_node: np.ndarray,      # Input pressure vector (flattened internal nodes)
    iicxn: int,              # Number of internal nodes in x
    iicyn: int,              # Number of internal nodes in y
    hplusx_elem: np.ndarray, # Coefficient array (ELEMENT-centered or face-centered)
    hplusy_elem: np.ndarray, # Coefficient array (ELEMENT-centered or face-centered)
    hcenter_node: np.ndarray,# Coefficient array (NODE-centered)
    oodx: float,             # 1 / dx
    oody: float,             # 1 / dy
    x_periodic: bool,        # Boolean or int flag
    y_periodic: bool,        # Boolean or int flag
    x_wall: bool,            # Boolean or int flag
    y_wall: bool,            # Boolean or int flag
    diag_inv_node: np.ndarray,# Diagonal preconditioner (NODE-centered)
    coriolis_elem: tuple,    # Tuple of ELEMENT-centered coeff arrays (cyy, cxx, cyx, cxy)
    out: np.ndarray = None   # Optional output array for in-place operation
) -> np.ndarray:
    """
    Calculates the action of a 2D discretized operator (Laplacian + Coriolis)
    on a vector 'p_node' using a 9-point stencil (matrix-free via Numba).

    Handles periodic and wall boundary conditions by adjusting coefficients.
    Operates on flattened arrays corresponding to internal grid nodes/elements.

    Args:
        p_node: Flattened 1D array of input values at internal nodes.
        iicxn, iicyn: Internal grid dimensions (number of nodes).
        hplusx_elem, hplusy_elem: Coefficient arrays, assumed mapped to elements
                                 (e.g., indexed by bottom-left node's internal index).
                                 Size should match number of internal elements.
        hcenter_node: Central coefficient at each internal node. Size ngnc.
        oodx, oody: Grid spacing reciprocals.
        x_periodic, y_periodic: Flags for periodic boundaries.
        x_wall, y_wall: Flags for wall boundaries (assumed zero Neumann).
        diag_inv_node: Diagonal preconditioning values at each internal node. Size ngnc.
        coriolis_elem: Tuple (cyy, cxx, cyx, cxy) of coefficient arrays,
                       assumed mapped to elements.
        out: Optional pre-allocated array (size ngnc) to store the result.

    Returns:
        A 1D numpy array (size ngnc) containing the result of the operator
        applied to p_node. If 'out' was provided, it returns 'out'.
    """
    ngnc = iicxn * iicyn # Total number of internal nodes

    # Initialize output array
    if out is None:
        lap_node = np.zeros(ngnc, dtype=p_node.dtype)
    else:
        # Ensure the output array has the correct shape if provided
        if out.shape[0] != ngnc:
            # Numba doesn't support raising exceptions easily, maybe return zeros or handle outside
            return np.zeros(ngnc, dtype=p_node.dtype) # Indicate error state
        lap_node = out
        lap_node[:] = 0.0 # Zero out array if accumulating isn't desired

    # Unpack Coriolis terms for clarity
    cyy_elem, cxx_elem, cyx_elem, cxy_elem = coriolis_elem

    # Use nested loops for better 2D clarity
    for j in range(iicyn):  # Loop over y index (internal nodes, 0 to iicyn-1)
        for i in range(iicxn):  # Loop over x index (internal nodes, 0 to iicxn-1)
            # --- Calculate current node index ---
            idx_mid = j * iicxn + i

            # --- Determine neighbor node coordinates (ni, nj) with periodic wrapping ---
            # Note: Python's % handles negative numbers correctly for wrapping in Numba too
            i_w = (i - 1 + iicxn) % iicxn if x_periodic else i - 1
            i_e = (i + 1) % iicxn         if x_periodic else i + 1
            j_s = (j - 1 + iicyn) % iicyn if y_periodic else j - 1
            j_n = (j + 1) % iicyn         if y_periodic else j + 1

            # --- Calculate flattened neighbor indices in 'p_node' array ---
            # These indices are potentially invalid if walls are present,
            # but we fetch p values first and apply BCs via coefficients later.
            idx_mm = idx_mid           # Middle-Middle (center)
            idx_wm = j * iicxn + i_w   # West-Middle
            idx_em = j * iicxn + i_e   # East-Middle
            idx_ms = j_s * iicxn + i   # Middle-South
            idx_mn = j_n * iicxn + i   # Middle-North
            idx_ws = j_s * iicxn + i_w # West-South
            idx_es = j_s * iicxn + i_e # East-South
            idx_wn = j_n * iicxn + i_w # West-North
            idx_en = j_n * iicxn + i_e # East-North

            # --- Boundary Checks for Non-Periodic Walls ---
            # These flags determine if a neighbor is *logically* outside a wall
            on_west_boundary  = (not x_periodic and i == 0)
            on_east_boundary  = (not x_periodic and i == iicxn - 1)
            on_south_boundary = (not y_periodic and j == 0)
            on_north_boundary = (not y_periodic and j == iicyn - 1)

            # --- Gather 'p' values from neighbors ---
            # For wall boundaries, fetch the value from the boundary node itself (p[idx_mid])
            # This effectively applies the Neumann BC when combined with zeroed coefficients later.
            p_mm = p_node[idx_mm]
            p_wm = p_node[idx_wm if not on_west_boundary else idx_mm]
            p_em = p_node[idx_em if not on_east_boundary else idx_mm]
            p_ms = p_node[idx_ms if not on_south_boundary else idx_mm]
            p_mn = p_node[idx_mn if not on_north_boundary else idx_mm]

            p_ws = p_node[idx_ws if not (on_west_boundary or on_south_boundary) else idx_mm]
            p_es = p_node[idx_es if not (on_east_boundary or on_south_boundary) else idx_mm]
            p_wn = p_node[idx_wn if not (on_west_boundary or on_north_boundary) else idx_mm]
            p_en = p_node[idx_en if not (on_east_boundary or on_north_boundary) else idx_mm]


            # --- Determine element indices for coefficients ---
            # Assuming coeffs arrays (hplus, coriolis) are mapped based on the
            # bottom-left node's *internal* index.
            # Calculate the potential indices of the 4 surrounding elements' bottom-left nodes.
            # Need to handle periodicity/walls for these indices if coeff arrays only store internal elements.
            elem_idx_sw = ((j - 1 + iicyn) % iicyn if y_periodic else j - 1) * iicxn + \
                          ((i - 1 + iicxn) % iicxn if x_periodic else i - 1)
            elem_idx_se = ((j - 1 + iicyn) % iicyn if y_periodic else j - 1) * iicxn + \
                          (i % iicxn if x_periodic else i) # Use i directly for SE/NE x-index
            elem_idx_nw = (j % iicyn if y_periodic else j) * iicxn + \
                          ((i - 1 + iicxn) % iicxn if x_periodic else i - 1)
            elem_idx_ne = (j % iicyn if y_periodic else j) * iicxn + \
                          (i % iicxn if x_periodic else i) # Current node (j,i) is bottom-left of NE element

            # --- Gather Coefficients (hplus, coriolis) ---
            # Use helper function to safely get coefficients, handling potential
            # out-of-bounds access for wall boundaries by returning 0.0

            # SW Element Coefficients
            valid_sw = (not (on_west_boundary or on_south_boundary))
            hpx_sw = hplusx_elem[elem_idx_sw] if valid_sw else 0.0
            hpy_sw = hplusy_elem[elem_idx_sw] if valid_sw else 0.0
            cxx_sw = cxx_elem[elem_idx_sw] if valid_sw else 0.0
            cyy_sw = cyy_elem[elem_idx_sw] if valid_sw else 0.0
            cxy_sw = cxy_elem[elem_idx_sw] if valid_sw else 0.0
            cyx_sw = cyx_elem[elem_idx_sw] if valid_sw else 0.0

            # SE Element Coefficients
            valid_se = (not (on_east_boundary or on_south_boundary))
            hpx_se = hplusx_elem[elem_idx_se] if valid_se else 0.0
            hpy_se = hplusy_elem[elem_idx_se] if valid_se else 0.0
            cxx_se = cxx_elem[elem_idx_se] if valid_se else 0.0
            cyy_se = cyy_elem[elem_idx_se] if valid_se else 0.0
            cxy_se = cxy_elem[elem_idx_se] if valid_se else 0.0
            cyx_se = cyx_elem[elem_idx_se] if valid_se else 0.0

            # NW Element Coefficients
            valid_nw = (not (on_west_boundary or on_north_boundary))
            hpx_nw = hplusx_elem[elem_idx_nw] if valid_nw else 0.0
            hpy_nw = hplusy_elem[elem_idx_nw] if valid_nw else 0.0
            cxx_nw = cxx_elem[elem_idx_nw] if valid_nw else 0.0
            cyy_nw = cyy_elem[elem_idx_nw] if valid_nw else 0.0
            cxy_nw = cxy_elem[elem_idx_nw] if valid_nw else 0.0
            cyx_nw = cyx_elem[elem_idx_nw] if valid_nw else 0.0

            # NE Element Coefficients
            valid_ne = (not (on_east_boundary or on_north_boundary))
            hpx_ne = hplusx_elem[elem_idx_ne] if valid_ne else 0.0
            hpy_ne = hplusy_elem[elem_idx_ne] if valid_ne else 0.0
            cxx_ne = cxx_elem[elem_idx_ne] if valid_ne else 0.0
            cyy_ne = cyy_elem[elem_idx_ne] if valid_ne else 0.0
            cxy_ne = cxy_elem[elem_idx_ne] if valid_ne else 0.0
            cyx_ne = cyx_elem[elem_idx_ne] if valid_ne else 0.0

            # --- Calculate fluxes / derivatives (using gathered p and coeffs) ---
            # These coefficients (hpx_*, cxx_*, etc.) are now correctly zeroed
            # if they correspond to an element outside a wall boundary.
            Dx_tl = HALF * (p_mn - p_wn + p_mm - p_wm) * hpx_nw # Top-Left face flux approx
            Dx_tr = HALF * (p_en - p_mn + p_em - p_mm) * hpx_ne # Top-Right face flux approx
            Dx_bl = HALF * (p_mm - p_wm + p_ms - p_ws) * hpx_sw # Bottom-Left face flux approx
            Dx_br = HALF * (p_em - p_mm + p_es - p_ms) * hpx_se # Bottom-Right face flux approx

            Dy_tl = HALF * (p_mm - p_mn + p_wm - p_wn) * hpy_nw # Top-Left face flux approx
            Dy_tr = HALF * (p_em - p_en + p_mm - p_mn) * hpy_ne # Top-Right face flux approx
            Dy_bl = HALF * (p_ms - p_mm + p_ws - p_wm) * hpy_sw # Bottom-Left face flux approx
            Dy_br = HALF * (p_es - p_em + p_ms - p_mm) * hpy_se # Bottom-Right face flux approx

            # --- Combine terms for the operator action at 'idx_mid' ---
            # Based on divergence form: d/dx(F_x) + d/dy(F_y)
            # F_x approx = cxx * Dx + cxy * Dy
            # F_y approx = cyx * Dx + cyy * Dy
            # d/dx(F_x) approx = ( F_x_east_face - F_x_west_face ) * oodx
            # d/dy(F_y) approx = ( F_y_north_face - F_y_south_face ) * oody

            # Approximate fluxes at cell faces around node (i,j)
            Fx_west = HALF * (cxx_sw * Dx_bl + cxx_nw * Dx_tl + cxy_sw * Dy_bl + cxy_nw * Dy_tl)
            Fx_east = HALF * (cxx_se * Dx_br + cxx_ne * Dx_tr + cxy_se * Dy_br + cxy_ne * Dy_tr)
            Fy_south = HALF * (cyx_sw * Dx_bl + cyx_se * Dx_br + cyy_sw * Dy_bl + cyy_se * Dy_br)
            Fy_north = HALF * (cyx_nw * Dx_tl + cyx_ne * Dx_tr + cyy_nw * Dy_tl + cyy_ne * Dy_tr)

            divergence = (Fx_east - Fx_west) * oodx + (Fy_north - Fy_south) * oody

            # --- Final value for the node ---
            # Check if the original formula includes the hcenter term *before* or *after* divergence.
            # Assuming it's added:
            lap_val = divergence + hcenter_node[idx_mid] * p_mm

            # Apply diagonal preconditioner
            lap_node[idx_mid] = lap_val * diag_inv_node[idx_mid]

    return lap_node

In [15]:
import numpy as np
import time
import sys
#
# # --- Assume functions are defined/imported ---
# # Make sure these placeholder functions have the EXACT same signature
# # as your actual functions. Replace the imports/definitions below.
#
# # Placeholder for the ORIGINAL Numba function
# try:
#     # Option 1: If it's in a file named, e.g., original_laplacian.py
#     from original_laplacian import lap2D_gather as lap2D_gather_original
#     print("Imported lap2D_gather_original from original_laplacian.py")
# except ImportError:
#     print("Warning: Could not import lap2D_gather_original. Using placeholder.")
#     # Option 2: Define a dummy placeholder if the real one isn't available
#     # This will likely fail comparisons but allows the script structure to run.
#     import numba as nb
#     @nb.jit(nopython=True)
#     def lap2D_gather_original(p, igx,igy, iicxn, iicyn, hplusx, hplusy, hcenter, oodx, oody, x_periodic, y_periodic, x_wall, y_wall, diag_inv, coriolis):
#         # This placeholder just returns zeros - REPLACE IT with your original code
#         # Note: The signature must match EXACTLY, including parameter names if
#         # the original code relied on them implicitly (though less likely in Numba).
#         # The dummy 'igx', 'igy' are included here assuming they were in the original
#         # even if unused, to maintain the signature for the call.
#         ngnc = iicxn * iicyn
#         return np.zeros(ngnc, dtype=p.dtype)
#
# # Placeholder for the NEW Refactored Numba function
# try:
#     # Option 1: If it's in a file named, e.g., refactored_laplacian.py
#     from refactored_laplacian import lap2D_gather_refactored_numba
#     print("Imported lap2D_gather_refactored_numba from refactored_laplacian.py")
# except ImportError:
#     print("Warning: Could not import lap2D_gather_refactored_numba. Using placeholder.")
#     # Option 2: Define a dummy placeholder
#     import numba as nb
#     @nb.jit(nopython=True)
#     def lap2D_gather_refactored_numba(p_node: np.ndarray, iicxn: int, iicyn: int, hplusx_elem: np.ndarray, hplusy_elem: np.ndarray, hcenter_node: np.ndarray, oodx: float, oody: float, x_periodic: bool, y_periodic: bool, x_wall: bool, y_wall: bool, diag_inv_node: np.ndarray, coriolis_elem: tuple, out: np.ndarray = None):
#         # This placeholder just returns zeros - REPLACE IT with your refactored code
#         ngnc = iicxn * iicyn
#         if out is None:
#             out = np.zeros(ngnc, dtype=p_node.dtype)
#         else:
#             out[:] = 0.0
#         return out
# # --- End Placeholder functions ---


def run_numba_comparison(iicxn, iicyn, num_trials=10):
    """
    Compares the original and refactored Numba implementations.
    """
    print("-" * 60)
    print(f"Running comparison for grid size: {iicxn} x {iicyn}")
    print(f"Number of trials for timing: {num_trials}")
    print("-" * 60)

    ngnc = iicxn * iicyn # Number of internal grid nodes
    nelem = iicxn * iicyn # Assuming element count matches internal nodes
    # Dummy ghost cell info if needed by original signature (adjust if different)
    igx, igy = 2, 2

    # --- Generate Consistent Random Test Data (CPU) ---
    np.random.seed(42)
    p_np = np.random.rand(ngnc).astype(np.float64)
    hplusx_np = np.random.rand(nelem).astype(np.float64) + 0.1
    hplusy_np = np.random.rand(nelem).astype(np.float64) + 0.1
    hcenter_np = np.random.rand(ngnc).astype(np.float64) * 5.0 + 1.0
    diag_inv_np = (1.0 / hcenter_np).astype(np.float64)
    coriolis_np = tuple([ (np.random.rand(nelem).astype(np.float64) - 0.5) * 0.1 for _ in range(4) ])

    oodx = float(iicxn)
    oody = float(iicyn)
    x_periodic, y_periodic = True, True
    x_wall, y_wall = False, False

    # --- 1. Run Original Numba Version ---
    print("Running Original Numba version...")
    try:
        # Warm-up call
        _ = lap2D_gather(p_np, igx, igy, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
                         x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np)

        start_time_orig = time.perf_counter()
        for _ in range(num_trials):
            result_orig = lap2D_gather(p_np, igx, igy, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
                                     x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np)
        end_time_orig = time.perf_counter()
        time_orig_avg = (end_time_orig - start_time_orig) / num_trials
        print(f"Original average execution time: {time_orig_avg:.6f} seconds")
    except Exception as e:
        print(f"Error running original version: {e}")
        time_orig_avg = float('inf')
        result_orig = None


    # --- 2. Run Refactored Numba Version ---
    print("\nRunning Refactored Numba version...")
    try:
        # Optional output buffer for refactored version
        out_buffer = np.zeros_like(p_np)
        # Warm-up call
        _ = lap2D_gather_refactored_numba(p_np, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
                                x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np, out=out_buffer)

        start_time_ref = time.perf_counter()
        for _ in range(num_trials):
            # Example using the output buffer
            result_ref = lap2D_gather_refactored_numba(p_np, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
                                           x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np, out=out_buffer)
            # Or call without 'out' if preferred:
            # result_ref = lap2D_gather_refactored_numba(p_np, iicxn, iicyn, hplusx_np, hplusy_np, hcenter_np, oodx, oody,
            #                                x_periodic, y_periodic, x_wall, y_wall, diag_inv_np, coriolis_np)
        end_time_ref = time.perf_counter()
        time_ref_avg = (end_time_ref - start_time_ref) / num_trials
        print(f"Refactored average execution time: {time_ref_avg:.6f} seconds")
        # If using 'out', result_ref is the same buffer 'out_buffer'
        if 'out_buffer' in locals() and result_ref is out_buffer:
             result_ref = out_buffer.copy() # Copy if needed for comparison, as buffer is overwritten

    except Exception as e:
        print(f"Error running refactored version: {e}")
        time_ref_avg = float('inf')
        result_ref = None

    # --- 3. Compare Results ---
    print("\nComparing results...")
    if result_orig is not None and result_ref is not None:
        try:
            are_close = np.allclose(result_orig, result_ref, rtol=1e-9, atol=1e-12) # Use tighter tolerance
            print(f"Results are close: {are_close}")
            if not are_close:
                diff = np.abs(result_orig - result_ref)
                max_abs_diff = np.max(diff)
                # Calculate relative difference carefully
                # Add small epsilon to denominator to avoid division by zero/large rel diff for small numbers
                denom = np.maximum(np.abs(result_orig), np.abs(result_ref)) + 1e-15
                max_rel_diff = np.max(diff / denom)

                print(f"  Max absolute difference: {max_abs_diff:.2e}")
                print(f"  Max relative difference: {max_rel_diff:.2e}")
                # Optionally print indices where differences occur
                # diff_indices = np.where(~np.isclose(result_orig, result_ref, rtol=1e-9, atol=1e-12))[0]
                # print(f"  Indices with differences (first few): {diff_indices[:10]}")
        except Exception as e:
            print(f"Error during comparison: {e}")
            print(f"  Original result shape: {result_orig.shape}, dtype: {result_orig.dtype}")
            print(f"  Refactored result shape: {result_ref.shape}, dtype: {result_ref.dtype}")
    else:
        print("Comparison skipped due to missing results.")

    print("-" * 60)
    return time_orig_avg, time_ref_avg


if __name__ == "__main__":
    # --- Configuration ---
    grid_sizes = [(32, 32), (64, 64), (128, 128), (256, 256)] # Test different sizes
    trials = 100 # Number of repetitions for timing

    # --- Run Comparisons ---
    results = {}
    for size in grid_sizes:
        iicxn, iicyn = size
        try:
            orig_t, ref_t = run_numba_comparison(iicxn, iicyn, num_trials=trials)
            results[size] = {'original_time': orig_t, 'refactored_time': ref_t}
        except Exception as e:
            print(f"\n!!!!!! Error during comparison for size {size}: {e} !!!!!!\n")
            results[size] = {'original_time': float('inf'), 'refactored_time': float('inf')}


    # --- Print Summary ---
    print("\n" + "=" * 60)
    print("Numba Comparison Summary (Average Times in seconds)")
    print("=" * 60)
    print(f"{'Grid Size':<12} | {'Original Time':<18} | {'Refactored Time':<18} | {'Speedup (Ref/Orig)':<18}")
    print("-" * 60)
    for size, times in results.items():
        orig_t = times['original_time']
        ref_t = times['refactored_time']
        if orig_t == 0 or orig_t == float('inf') or ref_t == float('inf'):
            speedup_str = "N/A"
        else:
            speedup = orig_t / ref_t
            speedup_str = f"{speedup:.2f}x"
        print(f"{f'{size[0]}x{size[1]}':<12} | {orig_t:.6f}{'s':<12} | {ref_t:.6f}{'s':<12} | {speedup_str:<18}")
    print("=" * 60)

------------------------------------------------------------
Running comparison for grid size: 32 x 32
Number of trials for timing: 100
------------------------------------------------------------
Running Original Numba version...
Original average execution time: 0.000053 seconds

Running Refactored Numba version...
Error running refactored version: name 'lap2D_gather_refactored_numba' is not defined

Comparing results...
Comparison skipped due to missing results.
------------------------------------------------------------
------------------------------------------------------------
Running comparison for grid size: 64 x 64
Number of trials for timing: 100
------------------------------------------------------------
Running Original Numba version...
Original average execution time: 0.000208 seconds

Running Refactored Numba version...
Error running refactored version: name 'lap2D_gather_refactored_numba' is not defined

Comparing results...
Comparison skipped due to missing results.
-

In [ ]:
import numpy as np

def gradient_nodal_to_interface(p_nodal: np.ndarray,
                                dx: float, dy: float,
                                nx: int, ny: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Computes the gradient of a nodal scalar field, evaluating components
    at the corresponding interface centers.

    Args:
        p_nodal (np.ndarray): Scalar field defined on nodes, shape (nx+1, ny+1).
                              Assumes boundary values are appropriately set.
        dx (float): Grid spacing in x.
        dy (float): Grid spacing in y.
        nx (int): Number of cells (inner nodes) in x direction.
        ny (int): Number of cells (inner nodes) in y direction.

    Returns:
        tuple[np.ndarray, np.ndarray]:
            - grad_x_at_x_interfaces: dP/dx evaluated at vertical (x) interfaces,
                                      shape (nx, ny+1). Index (i, j) corresponds
                                      to interface between nodes (i,j) and (i+1,j).
            - grad_y_at_y_interfaces: dP/dy evaluated at horizontal (y) interfaces,
                                      shape (nx+1, ny). Index (i, j) corresponds
                                      to interface between nodes (i,j) and (i,j+1).
    """
    if p_nodal.shape != (nx + 1, ny + 1):
        raise ValueError(f"Input p_nodal shape {p_nodal.shape} does not match expected ({nx+1}, {ny+1})")

    # Calculate dP/dx at vertical interfaces (between columns)
    # Difference along axis=0 (x-direction)
    grad_x_at_x_interfaces = (p_nodal[1:, :] - p_nodal[:-1, :]) / dx

    # Calculate dP/dy at horizontal interfaces (between rows)
    # Difference along axis=1 (y-direction)
    grad_y_at_y_interfaces = (p_nodal[:, 1:] - p_nodal[:, :-1]) / dy

    # Check shapes
    if grad_x_at_x_interfaces.shape != (nx, ny + 1):
         raise RuntimeError(f"Internal error: grad_x shape {grad_x_at_x_interfaces.shape} unexpected.")
    if grad_y_at_y_interfaces.shape != (nx + 1, ny):
         raise RuntimeError(f"Internal error: grad_y shape {grad_y_at_y_interfaces.shape} unexpected.")

    return grad_x_at_x_interfaces, grad_y_at_y_interfaces

In [ ]:
import numpy as np

def divergence_interface_to_nodal(flux_x_at_x_interfaces: np.ndarray,
                                  flux_y_at_y_interfaces: np.ndarray,
                                  dx: float, dy: float,
                                  nx: int, ny: int) -> np.ndarray:
    """
    Computes the divergence of a vector field F, given its normal components
    defined on the interface centers. The divergence is evaluated at the
    inner nodes.

    This implements Div(F) = d(Fx)/dx + d(Fy)/dy using centered differences
    based on interface fluxes.

    Args:
        flux_x_at_x_interfaces (np.ndarray): Fx component evaluated at vertical
                                            (x) interfaces, shape (nx, ny+1).
                                            Index (i, j) is flux between nodes
                                            (i,j) and (i+1,j).
        flux_y_at_y_interfaces (np.ndarray): Fy component evaluated at horizontal
                                            (y) interfaces, shape (nx+1, ny).
                                            Index (i, j) is flux between nodes
                                            (i,j) and (i,j+1).
        dx (float): Grid spacing in x.
        dy (float): Grid spacing in y.
        nx (int): Number of cells (inner nodes) in x direction.
        ny (int): Number of cells (inner nodes) in y direction.

    Returns:
        np.ndarray: Divergence evaluated at inner nodes, shape (nx, ny).
                    Index (i,j) corresponds to inner node originally at
                    p_nodal[i+1, j+1].
    """
    if flux_x_at_x_interfaces.shape != (nx, ny + 1):
         raise ValueError(f"Input flux_x shape {flux_x_at_x_interfaces.shape} does not match expected ({nx}, {ny+1})")
    if flux_y_at_y_interfaces.shape != (nx + 1, ny):
         raise ValueError(f"Input flux_y shape {flux_y_at_y_interfaces.shape} does not match expected ({nx+1}, {ny})")

    # Calculate d(Fx)/dx at nodes (i+1, j+1) for i=0..nx-1, j=0..ny-1
    # Requires Fx at interfaces (i+1/2, j+1) and (i+1/2, j) -> Fx[i, j+1]
    # and Fx at interfaces (i-1/2, j+1) and (i-1/2, j) -> Fx[i-1, j+1]
    # The indices need careful thought. Let's target node (I, J) where I=1..nx, J=1..ny
    # Node (I, J) is surrounded by x-interfaces at index (I-1, J) and (I, J)
    # and y-interfaces at index (I, J-1) and (I, J).

    # Difference Fx across the node in x-direction
    # flux_x_at_x_interfaces[I, J] is the flux on the right interface of node (I,J)
    # flux_x_at_x_interfaces[I-1, J] is the flux on the left interface of node (I,J)
    # We need these for J=1..ny (inner nodes plus boundaries in y)
    dFx_dx_at_nodes = (flux_x_at_x_interfaces[:, 1:] - flux_x_at_x_interfaces[:, :-1]) / dx

    # Difference Fy across the node in y-direction
    # flux_y_at_y_interfaces[I, J] is the flux on the top interface of node (I,J)
    # flux_y_at_y_interfaces[I, J-1] is the flux on the bottom interface of node (I,J)
    # We need these for I=1..nx (inner nodes plus boundaries in x)
    dFy_dy_at_nodes = (flux_y_at_y_interfaces[1:, :] - flux_y_at_y_interfaces[:-1, :]) / dy

    # Check shapes - these differences should be on the inner nodes grid
    if dFx_dx_at_nodes.shape != (nx, ny):
        raise RuntimeError(f"Internal error: dFx_dx shape {dFx_dx_at_nodes.shape} unexpected.")
    if dFy_dy_at_nodes.shape != (nx, ny):
        raise RuntimeError(f"Internal error: dFy_dy shape {dFy_dy_at_nodes.shape} unexpected.")

    # Sum the partial derivatives to get the divergence
    divergence_at_inner_nodes = dFx_dx_at_nodes + dFy_dy_at_nodes

    return divergence_at_inner_nodes

In [ ]:
import numpy as np

def calculate_flux_on_interfaces(
    grad_x_at_x_interfaces: np.ndarray,
    grad_y_at_y_interfaces: np.ndarray,
    C_tensor_at_interfaces: dict, # See details below
    nx: int, ny: int
) -> tuple[np.ndarray, np.ndarray]:
    """
    Calculates the vector flux F = C * grad(p) on the interfaces.

    Args:
        grad_x_at_x_interfaces (np.ndarray): dP/dx on x-interfaces, shape (nx, ny+1).
        grad_y_at_y_interfaces (np.ndarray): dP/dy on y-interfaces, shape (nx+1, ny).
        C_tensor_at_interfaces (dict): A dictionary containing the components
            of the tensor C evaluated AT THE INTERFACES. Expected keys:
            'cxx_x', 'cxy_x', 'cyx_x', 'cyy_x' (shape nx, ny+1) for x-interfaces.
            'cxx_y', 'cxy_y', 'cyx_y', 'cyy_y' (shape nx+1, ny) for y-interfaces.
            Calculating these C components accurately on interfaces from node/cell
            data is a separate (crucial) step involving averaging/interpolation.
        nx (int): Number of cells (inner nodes) in x direction.
        ny (int): Number of cells (inner nodes) in y direction.

    Returns:
        tuple[np.ndarray, np.ndarray]:
            - flux_x_at_x_interfaces: Fx component on x-interfaces, shape (nx, ny+1).
            - flux_y_at_y_interfaces: Fy component on y-interfaces, shape (nx+1, ny).
    """
    # Retrieve tensor components (assuming they exist and have correct shapes)
    cxx_x = C_tensor_at_interfaces['cxx_x'] # (nx, ny+1)
    cxy_x = C_tensor_at_interfaces['cxy_x'] # (nx, ny+1)
    cyx_x = C_tensor_at_interfaces['cyx_x'] # (nx, ny+1)
    cyy_x = C_tensor_at_interfaces['cyy_x'] # (nx, ny+1)

    cxx_y = C_tensor_at_interfaces['cxx_y'] # (nx+1, ny)
    cxy_y = C_tensor_at_interfaces['cxy_y'] # (nx+1, ny)
    cyx_y = C_tensor_at_interfaces['cyx_y'] # (nx+1, ny)
    cyy_y = C_tensor_at_interfaces['cyy_y'] # (nx+1, ny)

    # We need grad_y at x-interfaces and grad_x at y-interfaces for the tensor product.
    # Interpolate using simple averaging.
    # Average grad_y[i,j] and grad_y[i+1,j] to get value at x-interface (i+1/2, j)
    # Note: grad_y has shape (nx+1, ny)
    grad_y_at_x_interfaces = 0.5 * (grad_y_at_y_interfaces[:-1, :] + grad_y_at_y_interfaces[1:, :]) # Shape (nx, ny)
    # Need shape (nx, ny+1) - pad y-boundary? Or special handling?
    # Let's assume for now we need values at all (nx, ny+1) x-interfaces.
    # Simple padding might be okay if boundaries are handled appropriately in C or grad.
    # Pad with nearest value (or zero, or extrapolate based on BCs)
    grad_y_at_x_interfaces_padded = np.pad(grad_y_at_x_interfaces, ((0,0),(1,0)), mode='edge') # Pad bottom row
    # Check if this padding makes sense physically based on the problem BCs!

    # Average grad_x[i,j] and grad_x[i,j+1] to get value at y-interface (i, j+1/2)
    # Note: grad_x has shape (nx, ny+1)
    grad_x_at_y_interfaces = 0.5 * (grad_x_at_x_interfaces[:, :-1] + grad_x_at_x_interfaces[:, 1:]) # Shape (nx, ny)
    # Need shape (nx+1, ny) - pad x-boundary?
    grad_x_at_y_interfaces_padded = np.pad(grad_x_at_y_interfaces, ((1,0),(0,0)), mode='edge') # Pad left col

    # Calculate Flux components on their respective interfaces
    # Fx = Cxx * dPdx + Cxy * dPdy (@ x-interfaces)
    flux_x = cxx_x * grad_x_at_x_interfaces + cxy_x * grad_y_at_x_interfaces_padded

    # Fy = Cyx * dPdx + Cyy * dPdy (@ y-interfaces)
    flux_y = cyx_y * grad_x_at_y_interfaces_padded + cyy_y * grad_y_at_y_interfaces

    return flux_x, flux_y


def laplacian_operator_action(p_nodal: np.ndarray,
                              C_tensor_at_interfaces: dict,
                              hcenter_inner: np.ndarray,
                              diag_inv_inner: np.ndarray,
                              dx: float, dy: float,
                              nx: int, ny: int) -> np.ndarray:
    """
    Computes the action of the anisotropic Helmholtz operator on p_nodal.
    Operator: diag_inv * [∇ ⋅ (C ∇p) + hcenter * p]
    Uses gradient_nodal_to_interface and divergence_interface_to_nodal.

    Args:
        p_nodal (np.ndarray): Nodal pressure field, shape (nx+1, ny+1).
        C_tensor_at_interfaces (dict): Tensor C components defined on interfaces.
                                     (See calculate_flux_on_interfaces docstring).
        hcenter_inner (np.ndarray): Helmholtz coefficient at inner nodes, shape (nx, ny).
        diag_inv_inner (np.ndarray): Final scaling factor at inner nodes, shape (nx, ny).
        dx, dy (float): Grid spacing.
        nx, ny (int): Inner dimensions.

    Returns:
        np.ndarray: Result of operator action on inner nodes, shape (nx, ny).
    """

    # 1. Calculate gradient components on interfaces
    grad_x, grad_y = gradient_nodal_to_interface(p_nodal, dx, dy, nx, ny)

    # 2. Calculate flux F = C * grad(p) on interfaces
    flux_x, flux_y = calculate_flux_on_interfaces(grad_x, grad_y, C_tensor_at_interfaces, nx, ny)

    # 3. Calculate divergence of the flux at inner nodes
    divergence_term = divergence_interface_to_nodal(flux_x, flux_y, dx, dy, nx, ny)

    # 4. Add Helmholtz term
    # Get p values at the inner nodes corresponding to the divergence output
    p_inner_nodes = p_nodal[1:nx+1, 1:ny+1]
    helmholtz_term = hcenter_inner * p_inner_nodes

    # 5. Combine and apply diagonal scaling
    lap_result = divergence_term + helmholtz_term
    final_result = lap_result * diag_inv_inner # Element-wise multiplication

    return final_result